In [2]:
# import libraries
import pandas as pd
import numpy as np
import glob, os
data='../data/'

In [3]:
# download data
hip=pd.read_csv(data+'hipsci_files.tsv',sep='\t',index_col=0) # hipsci_files.tsv file is exported from https://www.hipsci.org/lines/#/lines 
ftps=hip['FTP url'].values
n=1                           # set n equal to 330 to download all the data 
for ftp in ftps[0:n]: 
    !wget -nc -P {data}/tpm {ftp+'*.tsv'}

--2021-04-16 15:54:42--  ftp://ftp.sra.ebi.ac.uk/vol1/ERZ267/ERZ267062/*.tsv
           => ‘../data//tpm/.listing’
Resolving ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)... 193.62.197.74
Connecting to ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)|193.62.197.74|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /vol1/ERZ267/ERZ267062 ... done.
==> PASV ... done.    ==> LIST ... done.

.listing                [  <=>               ]     330  1.15KB/s    in 0.3s    

2021-04-16 15:54:44 (1.15 KB/s) - ‘../data//tpm/.listing’ saved [330]

Removed ‘../data//tpm/.listing’.
File ‘../data//tpm/HPSI0114i-bezi_1.GRCh37.75.cdna.kallisto.transcripts.abundance.rnaseq.20150415.tsv’ already there; not retrieving.


In [4]:
# compine them into a list of list
ips=[]
cells=[]
for file in glob.glob(data+"tpm/*.tsv"):
    cells.append(file.split('.GRCh37.')[0].split('tpm/')[1])
    ip=pd.read_csv(file,sep='\t',index_col=0)[['tpm']].round(2)  
    ip=ip.tpm.values
    ips.append(ip)
ind=pd.read_csv(file,sep='\t',index_col=0).index

In [5]:
# convert it into pandas
df=pd.DataFrame(ips)
df.columns=ind
df.index=cells
df=df.T
df.shape

(179659, 8)

In [6]:
#get ENS to Gene mapping data and mapp
ens=pd.read_csv(data+'mart_export.txt.gz',sep='\t',index_col=1) # mart_export file exported from http://www.ensembl.org/biomart/martview/
ens=ens.rename(columns={'Gene name':'gene','Gene stable ID':'id'})
print(ens.shape)
df=df.merge(ens[['gene']],left_index=True,right_index=True)

(215170, 2)


In [8]:
#aggregate transcripts to get gene level data and write it into a combined table
dg=df.groupby('gene').sum()
dg.T.to_csv(data+'ips_tpm.csv.gz', compression='gzip')

In [9]:
#top few rows
dg.head()

,HPSI0114i-iisa_1,HPSI0114i-bezi_3,HPSI0114i-joxm_1,HPSI0114i-iisa_3,HPSI0114i-eipl_1,HPSI0114i-eipl_3,HPSI0114i-bezi_1,HPSI0114i-fikt_3
gene,,,,,,,,
A1BG,0.0,22.23,23.79,0.0,16.28,17.15,29.22,20.40
A1CF,0.0,0.05,0.07,0.0,0.06,0.02,0.05,0.05
A2M,0.0,0.47,0.94,0.0,9.33,1.52,0.58,0.28
A2ML1,0.0,2.14,9.94,0.0,2.91,2.25,2.80,4.28
A2MP1,0.0,0.45,0.05,0.0,0.29,0.25,0.32,0.27
